# This notebook explores valley bottom extraction methods

existing:

    - V-BET:
    - Geomorphons (WhiteBoxTools implementation)
    - CostAccumulation:
    - Slope Threshold:
    - Height Threshold:

totest:

- SurfaceVoronoi
- CostAccumulation delineation
- Adaptive threshold of slope and height

In [20]:
from dataclasses import dataclass
import glob
import os
import shutil

import leafmap.leafmap as leafmap
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import py3dep 
from pynhd import NHD
import rasterio as rio
from rasterio import Affine
from rasterio.plot import show
import whitebox

## Data Loading

In [2]:
@dataclass
class Raster:
    data: np.ndarray
    metadata: dict
    
    @classmethod
    def from_file(cls, raster_file):
        with rio.open(raster_file) as dataset:
            data = dataset.read(1)
            metadata = dataset.meta.copy()
                
            if "float" in str(data.dtype):
                metadata['nodata'] = np.nan if metadata['nodata'] is None else metadata['nodata']
                
                data[data > 3.0e38] = metadata['nodata']
                data[data < -3.0e38] = metadata['nodata']
                
            return cls(data, metadata)
        
    @classmethod
    def from_xarray_rio(cls, xarr):
        metadata = {
            'count': xarr.rio.count,
            'height': xarr.rio.height,
            'width': xarr.rio.width,
            'dtype': str(xarr.dtype),
            'crs': xarr.rio.crs,
            'nodata': xarr.rio.nodata,
            'transform': xarr.rio.transform()
        }
        data = xarr.values
        return cls(data, metadata)
        
    def to_file(self, output_file):
        with rio.open(output_file, 'w', **self.metadata) as dataset:
            dataset.write(self.data,1)
                
    def zero_to_nodata(self):
        self.data[self.data == 0] = self.metadata['nodata']
        self.data[self.data == 0] = self.metadata['nodata']
    
    def plot(self):
        show(self.data)        

In [3]:
gdf = gpd.read_file("../data/sample_valley_polygons/sample_valleys.geojson", driver='GeoJSON')
gdf = gdf[["layer", "geometry"]]
gdf = gdf.explode('geometry', index_parts=False)
gdf

,layer,geometry
0,russian_confined,"POLYGON ((6257387.991 1939912.155, 6254095.915..."
1,russian_large,"POLYGON ((6231409.111 1908836.263, 6231409.111..."
2,russian_med,"POLYGON ((6270261.608 1935260.309, 6270261.608..."
3,russian_partly_confined,"POLYGON ((6246509.827 1923216.026, 6246829.773..."
4,russian_small,"POLYGON ((6292388.399 1925695.607, 6292388.399..."
5,russian_unconfined,"POLYGON ((6273705.237 1945469.112, 6272021.311..."
6,santa_ynez_confined,"POLYGON ((7070336.506 552287.319, 7070800.663 ..."
7,santa_ynez_large,"POLYGON ((7071756.826 517740.130, 7071756.826 ..."
8,santa_ynez_med,"POLYGON ((7025749.607 485509.084, 7025749.607 ..."
9,santa_ynez_partly_confined,"POLYGON ((7082898.909 545930.692, 7083446.614 ..."


In [4]:
# get dems
def get_dem(geometry):
    # Fetch the DEM data using py3dep
    dem = py3dep.get_map("DEM", geometry, resolution=10, geo_crs="epsg:6418", crs="epsg:4326")
    dem.rio.reproject(6418)
    
    dem = Raster.from_xarray_rio(dem)
    
    return dem

dem = get_dem(gdf['geometry'][1])
gdf['dem'] = gdf['geometry'].apply(get_dem)
gdf

,layer,geometry,dem
0,russian_confined,"POLYGON ((6257387.991 1939912.155, 6254095.915...","Raster(data=array([[nan, nan, nan, ..., nan, n..."
1,russian_large,"POLYGON ((6231409.111 1908836.263, 6231409.111...","Raster(data=array([[nan, nan, nan, ..., nan, n..."
2,russian_med,"POLYGON ((6270261.608 1935260.309, 6270261.608...","Raster(data=array([[nan, nan, nan, ..., nan, n..."
3,russian_partly_confined,"POLYGON ((6246509.827 1923216.026, 6246829.773...","Raster(data=array([[ nan, nan, ..."
4,russian_small,"POLYGON ((6292388.399 1925695.607, 6292388.399...","Raster(data=array([[ nan, nan, ..."
5,russian_unconfined,"POLYGON ((6273705.237 1945469.112, 6272021.311...","Raster(data=array([[nan, nan, nan, ..., nan, n..."
6,santa_ynez_confined,"POLYGON ((7070336.506 552287.319, 7070800.663 ...","Raster(data=array([[nan, nan, nan, ..., nan, n..."
7,santa_ynez_large,"POLYGON ((7071756.826 517740.130, 7071756.826 ...","Raster(data=array([[nan, nan, nan, ..., nan, n..."
8,santa_ynez_med,"POLYGON ((7025749.607 485509.084, 7025749.607 ...","Raster(data=array([[nan, nan, nan, ..., nan, n..."
9,santa_ynez_partly_confined,"POLYGON ((7082898.909 545930.692, 7083446.614 ...","Raster(data=array([[ nan, nan, ..."


In [5]:
# get NHD flowlines
def get_flowlines(geometry):
    fields = [
        'COMID',
    ]

    nhd = NHD('flowline_mr', outfields = fields)
    try:
        flowlines = nhd.bygeom(geometry, geo_crs=6418)
        flowlines = flowlines.to_crs(6418)
    except Exception as e:
        flowlines = None
    return flowlines

gdf['flowlines'] = pd.Series([get_flowlines(geom) for geom in gdf['geometry']])
gdf

,layer,geometry,dem,flowlines
0,russian_confined,"POLYGON ((6257387.991 1939912.155, 6254095.915...","Raster(data=array([[nan, nan, nan, ..., nan, n...",ge...
1,russian_large,"POLYGON ((6231409.111 1908836.263, 6231409.111...","Raster(data=array([[nan, nan, nan, ..., nan, n...",...
2,russian_med,"POLYGON ((6270261.608 1935260.309, 6270261.608...","Raster(data=array([[nan, nan, nan, ..., nan, n...",g...
3,russian_partly_confined,"POLYGON ((6246509.827 1923216.026, 6246829.773...","Raster(data=array([[ nan, nan, ...",ge...
4,russian_small,"POLYGON ((6292388.399 1925695.607, 6292388.399...","Raster(data=array([[ nan, nan, ...",ge...
5,russian_unconfined,"POLYGON ((6273705.237 1945469.112, 6272021.311...","Raster(data=array([[nan, nan, nan, ..., nan, n...",ge...
6,santa_ynez_confined,"POLYGON ((7070336.506 552287.319, 7070800.663 ...","Raster(data=array([[nan, nan, nan, ..., nan, n...",None
7,santa_ynez_large,"POLYGON ((7071756.826 517740.130, 7071756.826 ...","Raster(data=array([[nan, nan, nan, ..., nan, n...",...
8,santa_ynez_med,"POLYGON ((7025749.607 485509.084, 7025749.607 ...","Raster(data=array([[nan, nan, nan, ..., nan, n...",...
9,santa_ynez_partly_confined,"POLYGON ((7082898.909 545930.692, 7083446.614 ...","Raster(data=array([[ nan, nan, ...",ge...


In [7]:
# all leafmap stuff seems to be in EPSG:4326
test = gdf.iloc[2]
m = leafmap.Map(center=[38.5, -122.95], zoom=12)

# add flowlines
m.add_gdf(test['flowlines'].to_crs("epsg:4326"), layer_name="flowlines")

# add raster
# write to temp file
meta = test['dem'].metadata.copy()
meta['count'] = 1
with rio.open("temp_dem.tif", 'w', **meta) as dataset:
    dataset.write(test['dem'].data,1)
m.add_raster("temp_dem.tif", colormap="terrain", layer_name="DEM")

m

Map(center=[38.5, -122.95], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

# WhiteBoxTools

In [21]:
wbt = whitebox.WhiteboxTools()
wbt.set_whitebox_dir("/Users/arthurkoehl/opt/WBT/")

working_dir = "/Users/arthurkoehl/programs/pasternack/valleys/wbt_workingdir/"
wbt.set_working_dir(working_dir)
wbt.version()

if os.path.isdir(working_dir):
    shutil.rmtree(working_dir)
os.mkdir(working_dir)

In [8]:
# geomorphons
print("class geomorphons ...")
dem = "low_relief_small.tif"
output = "geomorphons.tif"
wbt.geomorphons(
    dem, 
    output, 
    search=50, 
    threshold=0.0, 
    fdist=0, 
    skip=0, 
    forms=True, 
    residuals=False, 
)

class geomorphons ...
./whitebox_tools --run="Geomorphons" --wd="/Users/arthurkoehl/opt/WBT" --dem='low_relief_small.tif' --output='geomorphons.tif' --search=50 --threshold=0.0 --fdist=0 --skip=0 --forms -v --compress_rasters=False

****************************
* Welcome to Geomorphons   *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Generating global ternary codes...
Computing geomorphons...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
Progress: 

0

In [9]:
# valleyExtraction # this actually just extracts the channel network...

geomorphons_cmap = {
    Flat
2	Peak (summit)
3	Ridge
4	Shoulder
5	Spur (convex)
6	Slope
7	Hollow (concave)
8	Footslope
9	Valley
10	Pit (depression)
}

In [40]:
m = leafmap.Map(center=[38.5, -122.95], zoom=12)

m.add_gdf(test['flowlines'].to_crs("epsg:4326"), layer_name="flowlines")

# add raster
# write to temp file
m.add_raster("temp_dem.tif", cmap="terrain", layer_name="DEM")
m.add_raster("test.tif", layer_name="geomorphons", cmap="Pastel1")

m

Map(center=[38.5, -122.95], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…